In [ ]:
% load_ext autoreload
% autoreload 2



UsageError: Line magic function `%` not found.


In [ ]:

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import os.path

if not os.path.isfile('models.py'):
    from google.colab import drive
    drive.mount('/content/drive')

!pip install pandas_summary
#!pip install --upgrade pandas


https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [ ]:
import pandas as pd
import numpy as np
import datetime
#from pandas_summary import DataFrameSummary
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
import hyperopt
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [ ]:
if not os.path.isfile('models.py'):
    df = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/train_normalized_data2.fth')
    df_test = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/test_normalized_data2.fth')
else:
    df = pd.read_feather('train_normalized_data2.fth')
    df_test = pd.read_feather('test_normalized_data2.fth')

In [ ]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', #'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
#            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', #'SchoolHoliday_bw']

cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks',           'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']


cat_vars = ['Store']


In [ ]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']


contin_vars = ['CompetitionDistance', 
'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']   

# contin_vars = []

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
y_out_columns = ['Sales']

In [ ]:
# split_type = 'random'
split_type = 'no_split'
#split_type = 'last_week'

In [ ]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val, como venimos haciendo
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

In [ ]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [ ]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [ ]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [ ]:
def calculate_RMSE(X, y, log_output=True):
    y_preds = np.exp(model.predict(X, verbose=1)*max_log_y)
    return np.sqrt((((y - y_preds)/y)**2).sum()/len(y_preds))

In [ ]:
X_train.shape

(844338, 19)

In [ ]:
np.arange(1,64)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

## PRUEBA GUILLE


### CON BayesSearchCV

In [ ]:
#!pip install -U scikit-learn

!pip install scikit-learn==0.23.2
!pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.callbacks import CheckpointSaver
from skopt import load

In [ ]:
if not os.path.isfile('models.py'):
    checkpoint_saver = CheckpointSaver('./checkpoint.pkl',  compress=9)#'/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/checkpoint.pkl',  compress=9)    
else:
  checkpoint_saver = CheckpointSaver('checkpoint.pkl',  compress=9)    

In [ ]:
clf = LGBMRegressor(
        n_estimators=4000, n_jobs=-1)

In [ ]:
# Parametros para realizar el fit del modelo LGBMRegressor
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'l2', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            #'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [ ]:
fit_params

{'verbose': 100, 'feature_name': 'auto', 'categorical_feature': ['Store']}

In [ ]:
cargarRestoringCheck = True
if cargarRestoringCheck:
    res = load('checkpoint.pkl')
    print(res.x_iters)
    print(res.func_vals)


[[0.035253897966085836, 21, 13, 0.23801878101368126, 0.024568728553205476]]
[-0.73408649]


In [ ]:
#Parametros para realizar la busqueda en BayesSearchCV
param_bayes = {
        'max_depth': Integer(1,31),
        'learning_rate': Real(np.exp(-6), np.exp(-1), prior='log-uniform'),
        'reg_lambda': Real(np.exp(-8), np.exp(-3), prior='log-uniform'),
        'min_child_samples': Integer(1, 20),
        'min_child_weight': Real(np.exp(-8), np.exp(-1))  }
opt = BayesSearchCV(LGBMRegressor(n_estimators=4000, n_jobs=8), param_bayes,random_state=0, n_jobs=-1, verbose=1, cv=4)#fit_params=fit_params,  n_iter=4)

#Xtrain y yTrain no se ha spliteado, son todos los datos, no se necesita Train y Valid para hacer CV
fitBayesSearchCV = opt.fit(X_train, y_train.reshape(-1), callback=[checkpoint_saver])

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
#summarize results
print("Best: %f using %s" % (fitBayesSearchCV.best_score_, fitBayesSearchCV.best_params_))
means = fitBayesSearchCV.cv_results_['mean_test_score']
stds = fitBayesSearchCV.cv_results_['std_test_score']
params = fitBayesSearchCV.cv_results_['params']
print('--------------------------------------------------------------')
for mean, stdev, param in zip(means, stds, params):    
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
fitBayesSearchCV.best_params_

In [ ]:
modelBestBayesCVLGBMRegressor = LGBMRegressor(n_estimators=4000, n_jobs=8, learning_rate=0.3678794411714422, max_depth=27, min_child_samples=1, min_child_weight=0.36787944117144233, reg_lambda=0.0003354626279025119)

modelBestBayesCVLGBMRegressor.fit(X_train, y_train.reshape(-1), **fit_params)

In [ ]:
modelBestBayesCVLGBMRegressor.score(X_val, y_val)

## Segunda Opcion

In [ ]:
clf2 = LGBMRegressor(boosting_type='gbdt',
                                    class_weight='balanced',             
                         n_jobs=8, 
                         verbose=1)

search_spaces = {
        'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'num_leaves': Integer(2, 500),
        # 'max_depth': Integer(0, 500),
        # 'min_child_samples': Integer(0, 200), # minimal number of data in one leaf
        # 'max_bin': Integer(100, 100000), # max number of bins that feature values will be bucketed
        # 'subsample': Real(0.01, 1.0, 'uniform'),
        # 'subsample_freq': Integer(0, 10), # bagging fraction
        # 'colsample_bytree': Real(0.01, 1.0, 'uniform'), # enabler of bagging fraction
        # 'min_child_weight': Integer(0, 10), # minimal number of data in one leaf.
        # 'subsample_for_bin': Integer(100000, 500000), # number of data that sampled for histogram bins
        # 'reg_lambda': Real(1e-9, 1000, 'log-uniform'), # L2 regularization
        # 'reg_alpha': Real(1e-9, 1.0, 'log-uniform'), # L1 regularization
        # 'scale_pos_weight': Real(1e-6, 500, 'log-uniform'), 
        # 'n_estimators': Integer(10, 10000)        
        }                    

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
from sklearn.metrics import mean_squared_error
from time import time
from sklearn.metrics import make_scorer
from skopt.callbacks import EarlyStopper
from skopt.callbacks import DeadlineStopper # Stop the optimization before running out of a fixed budget of time.
from skopt.callbacks import VerboseCallback # Callback to control the verbosity
from skopt.callbacks import DeltaXStopper # Stop the optimization If the last two positions at which the objective has been evaluated are less than delta

#rms = mean_squared_error(y_actual, y_predicted, squared=False)


# Converting average precision score into a scorer suitable for model selection
score_mean_squared_error = make_scorer(mean_squared_error)

opt = BayesSearchCV(clf2,
                    search_spaces,
                    scoring=score_mean_squared_error,
                    cv=5,
                    n_iter=40,
                    n_jobs=-1,
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=22)
best_params = report_perf(opt, X_train, y_train.reshape(-1),'LightGBM')                    

In [ ]:
if log_output:
    y_pred_train = np.exp(modelBestBayesCVLGBMRegressor.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(modelBestBayesCVLGBMRegressor.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(modelBestBayesCVLGBMRegressor.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = modelBestBayesCVLGBMRegressor.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = modelBestBayesCVLGBMRegressor.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = modelBestBayesCVLGBMRegressor.predict(X_test, verbose=1)*y_std + y_mean

In [ ]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [ ]:
hasta aca ######

In [ ]:
def objective(params):
    #params = {
    #    #'max_depth': int(params['max_depth']),
    #    #'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    #    'learning_rate': '{:.4f}'.format(params['learning_rate'])
    #}
    
    clf = LGBMRegressor(
        n_estimators=4000, n_jobs=8,       
        **params
    )
    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()
    return score    

In [ ]:
space = {
    'max_depth': hp.choice('max_depth', list(np.arange(1, 31))),
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -8, -3),
    'min_child_samples': hp.choice('min_child_samples', list(np.arange(1, 20))),
    'min_child_weight': hp.loguniform('min_child_weight', -8, -1)    
}

tpe_trials = Trials()

bestGuille = fmin(fn=objective,
            space=space,
            trials = tpe_trials,
            verbose=1,
            algo=hyperopt.tpe.suggest,
            # algo=hyperopt.rand.suggest,
            return_argmin=False,
            max_evals=1000)

In [ ]:
tpe_results = pd.DataFrame({'loss': [x['loss'] for x in tpe_trials.results], 
                            'iteration': tpe_trials.idxs_vals[0]['max_depth'],
                            'max_depth': tpe_trials.idxs_vals[1]['max_depth']})
tpe_results.set_index('iteration')
tpe_results.loc[tpe_trials.idxs_vals[0]['learning_rate'], 'learning_rate']  = tpe_trials.idxs_vals[1]['learning_rate']
tpe_results.loc[tpe_trials.idxs_vals[0]['reg_lambda'], 'reg_lambda']  = tpe_trials.idxs_vals[1]['reg_lambda']
tpe_results.loc[tpe_trials.idxs_vals[0]['min_child_samples'], 'min_child_samples']  = tpe_trials.idxs_vals[1]['min_child_samples']
tpe_results.loc[tpe_trials.idxs_vals[0]['min_child_weight'], 'min_child_weight']  = tpe_trials.idxs_vals[1]['min_child_weight']
tpe_results.sort_values('loss').head(10)

In [ ]:
bestGuille

### FIN PRUEBA GUILLE

In [ ]:
space = {
    'max_depth': hp.choice('max_depth', list(np.arange(1,64))),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}
space

In [ ]:
def objective(params):
    #params = {
    #    #'max_depth': int(params['max_depth']),
    #    #'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    #    'learning_rate': '{:.4f}'.format(params['learning_rate'])
    #}
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5,
        **params
    )
    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()
    return score

In [ ]:
space = {
    #'max_depth': hp.quniform('max_depth', 1, 64, 4),
    #'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}

space = {
    'max_depth': hp.choice('max_depth', list(np.arange(1,64))),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

In [ ]:
best

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'learning_rate':[0.001, 0.05, 0.1, 0.5], 'max_depth':[1, 7, 31, 63]}
grid_search_res_1 = {'learning_rate': 0.05, 'min_child_samples': 5, 'max_depth': -1}
grid_search_res_2 = {'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20}

In [ ]:
min_child_samples = 5
n_estimators = 4000
learning_rate = 0.1
model = LGBMRegressor(n_estimators=n_estimators, **grid_search_res_2)

In [ ]:
clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=1, cv=3)

In [ ]:
fit_params={ "eval_metric" : 'l2', 
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars}
if split_type != 'no_split':
    fit_params["eval_set"] = [(X_val, y_val.reshape(-1))]
    fit_params["early_stopping_rounds"] = 100

In [ ]:
# clf.fit(X_train, y_train.reshape(-1), **fit_params)
model.fit(X_train, y_train.reshape(-1), **fit_params)

In [ ]:
clf

In [ ]:
# clf.best_params_

In [ ]:
# clf.best_score_

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
model.score(X_val, y_val)

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [ ]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [ ]:
calculate_RMSE(X_train, df_train['Sales'].values)

In [ ]:
calculate_RMSE(X_val, df_val['Sales'].values)

# Sumbit a la competición

In [ ]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_lightgbm_{split_type}-{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)